# 1일차: AI 에이전트의 기본 - `<think>`와 `<action>` 이해하기

**학습 목표:** 복잡한 딥러닝 없이, AI 에이전트가 어떻게 `상태`를 인식하고 `지시`를 받아 `생각(<think>)`하고 `행동(<action>)`하는지 그 핵심 로직을 이해합니다. 이를 통해 SFT(지도 미세조정)의 목표와 데이터셋의 역할을 명확히 파악합니다.

## 1. 시나리오 설정: 1차원 공간의 로봇

- **환경(Environment):** 0부터 10까지의 숫자로 이루어진 1차원 공간입니다.
- **로봇(Agent):** 이 공간의 특정 위치에 존재합니다.
- **상태(State):** 로봇의 현재 위치와 목표 지점의 위치를 텍스트로 표현합니다. (예: `"로봇은 2번 위치에 있고, 목표 지점은 7번입니다."`) 
- **지시(Instruction):** 로봇에게 내리는 명령입니다. (예: `"목표 지점으로 이동해."`) 
- **행동(Action):** 로봇이 취할 수 있는 행동은 `move(n)` 입니다. `n`은 이동할 칸 수를 의미하며, 양수면 오른쪽, 음수면 왼쪽으로 이동합니다.

## 2. 데이터셋 생성 (SFT를 위한 "전문가 시범" 만들기)

SFT의 핵심은 '정답'이 있는 데이터를 모델에 보여주는 것입니다. 여기서는 우리가 직접 '전문가'가 되어, 특정 상황과 지시가 주어졌을 때 가장 이상적인 생각과 행동이 무엇인지 정의합니다. 이것이 바로 AI를 위한 '교과서'가 됩니다.

In [ ]:
expert_demonstrations = [
    {
        "state": "Robot at position 2, goal is at 7",
        "instruction": "Move to the goal.",
        "expert_output": "<think>The robot is at 2 and the goal is at 7. I need to move 5 steps to the right.</think><action>move(5)</action>"
    },
    {
        "state": "Robot at position 8, goal is at 3",
        "instruction": "Go to the target location.",
        "expert_output": "<think>The robot is at 8, but the goal is at 3. I must move 5 steps to the left.</think><action>move(-5)</action>"
    },
    {
        "state": "Robot at position 5, goal is at 5",
        "instruction": "You have arrived. Stay put.",
        "expert_output": "<think>The robot is already at the goal position 5. No movement is necessary.</think><action>move(0)</action>"
    }
]

print("전문가 시범 데이터 (우리의 교과서):")
for i, demo in enumerate(expert_demonstrations):
    print(f"--- 예제 {i+1} ---")
    print(f"상황: {demo['state']}")
    print(f"지시: {demo['instruction']}")
    print(f"정답(생각+행동): {demo['expert_output']}")
    print()

## 3. "토이" 에이전트 (모델 시뮬레이션)

실제 LLM은 이 전문가 데이터를 학습하여 어떤 상황과 지시가 와도 `expert_output`과 유사한 결과를 생성하도록 훈련됩니다. 

여기서는 그 과정을 시뮬레이션하기 위해, 주어진 `state`와 `instruction`에 해당하는 정답을 데이터셋에서 그냥 '찾아오는' 간단한 함수를 만듭니다. 이것이 SFT가 완료된 모델이 해야 할 일입니다.

In [ ]:
def toy_agent(state, instruction):
    """SFT로 학습이 완료된 모델을 흉내 내는 함수.
    주어진 state와 instruction에 맞는 전문가의 행동을 찾아서 반환한다.
    """
    print(f"🤖 에이전트에게 전달된 정보:")
    print(f"  - 현재 상태: {state}")
    print(f"  - 받은 지시: {instruction}")
    
    for demo in expert_demonstrations:
        if demo["state"] == state and demo["instruction"] == instruction:
            print("✅ 전문가 데이터에서 일치하는 정답을 찾았습니다!")
            return demo["expert_output"]
    
    print("❌ 맞는 정답을 찾지 못했습니다...")
    return "<think>I don't know what to do.</think><action>move(0)</action>"


## 4. 환경(Environment)과 실행기(Executor) 만들기

에이전트가 `<action>`을 결정하면, 실제로 그 행동을 수행하고 결과를 확인할 '환경'이 필요합니다.

In [ ]:
import re

class RobotEnvironment:
    def __init__(self, robot_pos, goal_pos):
        self.robot_pos = robot_pos
        self.goal_pos = goal_pos
        print(f"환경이 설정되었습니다. 로봇 위치: {self.robot_pos}, 목표 위치: {self.goal_pos}")

    def get_state_text(self):
        return f"Robot at position {self.robot_pos}, goal is at {self.goal_pos}")
    
    def execute_action(self, action_string):
        """ '<action>move(5)</action>' 같은 문자열에서 행동을 파싱하고 실행합니다. """
        # 정규표현식을 사용해 <action> 태그 안의 내용을 추출합니다.
        match = re.search(r"<action>(.*)</action>", action_string)
        if not match:
            print("환경 오류: 행동(<action>)을 파싱할 수 없습니다.")
            return
        
        action_command = match.group(1) # 예: 'move(5)'
        
        # move(n) 명령어를 파싱합니다.
        move_match = re.search(r"move\((.*)\)", action_command)
        if not move_match:
            print(f"환경 오류: {action_command}는 알 수 없는 명령어입니다.")
            return
            
        value = int(move_match.group(1))
        print(f"🚀 행동 실행: {value}만큼 이동합니다.")
        self.robot_pos += value
        print(f"... 로봇의 새 위치는 {self.robot_pos} 입니다.")
        
    def check_success(self):
        if self.robot_pos == self.goal_pos:
            print(f"🎉 임무 성공! 로봇이 목표 지점 {self.goal_pos}에 도착했습니다.")
            return True
        else:
            print(f"🤔 임무 실패. 로봇은 {self.robot_pos}에 있고, 목표는 {self.goal_pos}입니다.")
            return False

## 5. 전체 프로세스 실행하기

이제 모든 조각을 맞춰서 에이전트의 전체 작동 흐름을 실행해 봅시다.

In [ ]:
# --- 시나리오 1 --- #
print("========== 시나리오 1 시작 ==========")
# 1. 환경 설정
env = RobotEnvironment(robot_pos=2, goal_pos=7)

# 2. 현재 상태와 지시 정의
current_state = env.get_state_text()
instruction = "Move to the goal."

# 3. 에이전트가 생각하고 행동 계획 생성
agent_output = toy_agent(current_state, instruction)
print(f"🧠 에이전트의 최종 응답:
{agent_output}")

# 4. 환경이 에이전트의 행동을 실행
env.execute_action(agent_output)

# 5. 결과 확인
env.check_success()
print("=====================================\n")

# --- 시나리오 2 --- #
print("========== 시나리오 2 시작 ==========")
env = RobotEnvironment(robot_pos=8, goal_pos=3)
current_state = env.get_state_text()
instruction = "Go to the target location."
agent_output = toy_agent(current_state, instruction)
print(f"🧠 에이전트의 최종 응답:
{agent_output}")
env.execute_action(agent_output)
env.check_success()
print("=====================================")

## 오늘의 학습 정리 및 다음 단계

축하합니다! 당신은 방금 AI 에이전트의 가장 기본적인 작동 원리를 직접 구현했습니다.

**오늘 우리가 한 것:**
1.  **데이터셋의 역할:** `expert_demonstrations`가 바로 SFT를 위한 '교과서'임을 확인했습니다.
2.  **모델의 목표:** `toy_agent` 함수가 바로 SFT가 끝난 모델이 해야 할 일, 즉 '교과서대로 따라하기'임을 이해했습니다.
3.  **`<think>`와 `<action>`의 중요성:** 모델의 출력을 '생각'과 '행동'으로 명확히 구분함으로써, 우리는 `execute_action` 함수에서 행동 부분만 정확히 골라내어 실행할 수 있었습니다. 이는 모델의 행동을 해석하고 제어하는 데 매우 중요합니다.

**다음 단계에서는...**

- **만약 `expert_demonstrations`에 없는 새로운 상황이 주어진다면?** 지금의 `toy_agent`는 대처할 수 없습니다. 바로 이 지점에서 **실제 LLM의 일반화(Generalization) 능력**이 필요합니다. 
- 다음 시간에는 **PEFT, QLoRA**와 같은 기술을 사용하여, 어떻게 거대한 언어 모델을 효율적으로 '학습'시켜서, 우리가 만든 '교과서'에 없는 새로운 문제도 잘 풀도록 만들 수 있는지 알아볼 것입니다.